## Obtain Credentials (not everytime)

Google Earth Engine (GEE) periodically requests users to renew their credentials by calling the 'ee.Authenticate()' function. Upon calling this function, a web page will open where you can sign in with your Google account. This step allows GEE to authenticate your identity and associate your API requests with your Google account. After the authentication process is successfully completed, a persistent token will be stored on your local machine (valid for a few days). 

In [3]:
import ee
ee.Authenticate()

Enter verification code:  4/1AfJohXmY7jcTG98dKqs1sbqjSEs54KzfN0yA3odYHHi1mWsw1cEOWS5o7To



Successfully saved authorization token.


## Initialize with Regular Google Account

This is normally the start point to execute the Mosaic Tool. By calling the 'ee.Initialize()' function, the saved credentials will be automatically retrieved, allowing you to make requests through the GEE's Python API.

In [1]:
import ee
ee.Initialize()

import geemap
import Image
import Mosaic

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


## Define Input Parameters

To run the Mosaic Tool, a set of parameters must be supplied. To simplify the provision of these parameters, a Python dictionary is utilized as a container to hold 12 “key:value” pairs, with the last three being optional. Each parameter is outlined as follows:
   
(1) Value for 'sensor' key: a string denoting the sensor type and data unit. Currently, valid values include 'L8_SR', 'S2_SR', 'L8_TOA' and 'S2_TOA', which denote Landsat 8/9 and Sentinel-2 surface reflectance and TOA reflectance data, respectively. 

(2) Value for 'year' key: a 4-digits integer representing the acquisition year (e.g., 2020) of the daily images used in the compositing.

(3) Value for 'months' key: a list of integers specifying the months of the year (e.g., [6, 7] for June and July). With a list of month numbers, the Mosaic Tool can produce multiple monthly composite images in a single execution. To generate a composite image for a peak season 
(June 15 to September 15) of a year (defined by the value of the 'year' key), simply include a negative integer in the list.

(4) Value for 'tile_names' key: a set of strings representing the names of grid tiles, each covering a 900km x 900km area. By providing a list of tile names, the Mosaic Tool can create composite images for multiple tiles in a single execution. To generate a composite image for a customized region, include an additional "key:value" pair in this parameter dictionary (see the value specification for 'custom_region' key. 

(5) Value for 'spatial_scale' key: an integer defining the spatial resolution (in meters) for exporting the resultant composite images (e.g., 30 and 20 for Landsat and Sentinel-2 composites, respectively).

(6) Value for 'out_location' key: a string specifying the destination for exporting the resultant composites. Valid values are 'drive' for Google Drive (GD) or 'storage' for Google Cloud Storage (GCS).

(7) Value for 'GCS_bucket' key: a string indicating a bucket name on GCS. This parameter is necessary only when the exporting destination is GCS ('storage' for 'out_location' key). Note that the specified bucket must exist on GCS before exporting. 

(8) Value for 'out_folder' key: the folder name on GD or GCS for exporting composites. If you prefer not to export all the composites to the same directory, leave an empty string for this key. The Tool will then automatically create separate folders for the composites of different tiles according to tile name and acquisition year.

(9) Value for 'custom_region' key: an "ee.Geometry" object created using the "ee.Geometry.Polygon()" function, which takes a list of Latitude and Longitude coordinates as inputs. Include this 'key:value' pair only when a customized region has to be defined. Otherwise, DO NOT include this 'key:value' pair in the parameter dictionary, as it will overwrite the values for the 'tile_names' key. 

(10) Value 'start_date' key: a string (e.g., '2022-06-15') to specify the start date of a customized compositing period. 

(11) Value for 'end_date' key: a string (e.g., '2022-09-15') to specify the end date of a custmoized compositing period. Please note that the strings for 'start_date' and 'end_date' keys should be omitted from this parameter dictionary unless a user-defined compositing period needs to be specified.

(12) value for 'projection' key: a string specifying a customized projection. The default projection used in the LEAF Tool is 'EPSG:3979'.


Among the 12 input parameters, 2 keys ('months' and 'tile_names') require list inputs. By combining these lists differently, you can handle various scenarios for producing composites. For instance, by providing [7, 8] and ['tile41', 'tile42', 'tile43']) to 'months' and 'tile_names' keys, respectively, you can create July and August monthly composites for each of the three tiles.

In [4]:
user_region = ee.Geometry.Polygon([[-76.12016546887865,45.183832177265906], [-75.38339483899584,45.170763450281996],
                                     [-75.39026129407397,45.5639242833682], [-76.10505926770678,45.56776998764525], 
                                     [-76.12016546887865,45.183832177265906]])

In [ ]:
params =  {
    'sensor': 'S2_SR',            # A sensor type string (e.g., 'S2_SR' or 'L8_SR')
    'year': 2022,                 # An integer representing image acquisition year
    'nbYears': -1,                 # positive int for annual product, or negative int for monthly product
    'months': [7,8],                # A list of integers represening one or multiple monthes     
    'tile_names': ['tile55'],       # A list of (sub-)tile names (defined using CCRS' tile griding system) 
    'prod_names': ['mosaic', 'date'],  
    'out_location': 'drive',      # Exporting location ('drive', 'storage' or 'asset') 
    'resolution': 20,              # Exporting spatial resolution
    'GCS_bucket': 's2_mosaic_2020',   # An unique bucket name on Google Cloud Storage
    'out_folder': 'test_mosaic_tool', # the folder name for exporting
    'extra_bands': Image.EXTRA_ANGLE, # Image.EXTRA_ANGLE and Image.EXTRA_NONE represent with and without imaging angles in output, respectively 
    #'custom_region': user_region, # A given user-defined region. Only include this 'key:value' pair as necessary
    #'start_date': '2022-06-15',   # A string for specifying the start date of a customized compositing period.
    #'end_date': '2022-09-15',     # A string for specifying the end date of a customized compositing period.
}

## Generate and Export Compositing Products

In [ ]:
mosaic = Mosaic.Mosaic_production(params, False)

<valid_user_params> 'spatial_scale' key in given parameter dictionary is invalid!


AttributeError: module 'eoTileGrids' has no attribute 'valid_tile_name'

## Check the Status of Exporting Tasks (optional)

Depending on whether you are using a regular Google account or a Google Service account to operate Google Earth Engine, there are two methods available to monitor the status of exporting tasks. If you're using a regular Google account, you can check the status by either accessing the "Tasks" tab located in the right panel of your Earth Engine Code Editor or executing the code provided in the subsequent cell. However, if you're using a Google Service account, the only way to monitor the status of exporting tasks is by running the code in the following cell.

The function "manage_tasks" requires two input parameters. The first parameter is a string and can take either 'status' or 'cancel' as its value. 'status' is used to check the status of exporting tasks, while 'cancel' is used to cancel all the exporting tasks. The second parameter is a string used to filter task names. For instance, providing '2019' will retrieve tasks with names that include '2019'. 

In [4]:
Image.manage_tasks('cancel', '2020')

tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m 